# PyCity Schools Analysis

In [ ]:
# Dependencies and Setup
import pandas as pd

# File to Load (** SOMETHING NEEDS TO CHANGE HERE? Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

## District Summary

In [ ]:
school_data_complete.describe()

In [ ]:
#Total number of unique schools
total_unique_schools = school_data_complete["school_name"].nunique()
total_unique_schools

In [ ]:
#Total students
individual_students = school_data_complete["Student ID"].count()
individual_students

In [ ]:
#Total budget
unique_schools = school_data_complete.drop_duplicates(subset="school_name")
TBudget = unique_schools["budget"].sum()
TBudget

In [ ]:
#Average math score
AvgMathS = school_data_complete["math_score"].mean()
AvgMathS

In [ ]:
#Average reading score
AvgReadS = school_data_complete["reading_score"].mean()
AvgReadS

In [ ]:
#% passing math (the percentage of students who passed math)
PassWMath = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
PassWMath_Percent = PassWMath / float(individual_students) * 100
PassWMath_Percent

In [ ]:
#% passing reading (the percentage of students who passed reading)
PassWRead = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
PassWRead_Percent = PassWRead / float(individual_students) * 100
PassWRead_Percent

In [ ]:
# overall passing (the percentage of students who passed math AND reading)
OverAllPassCount = school_data_complete[(school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
OverallPRate = OverAllPassCount /  float(individual_students) * 100
OverallPRate

In [ ]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame(
{
    "Total School": [total_unique_schools],
    "Total Students": [individual_students],
    "Total Budget": [TBudget],
    "Average Math Score": [AvgMathS],
    "Average Reading Score": [AvgReadS], 
    "% Passing Math": [PassWMath_Percent],
    "% Passing Reading": [PassWRead_Percent],
    "% Overall Passing": [OverallPRate]
}
)

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary

## School Summary

In [ ]:
#School type
school_types = unique_schools.set_index(["school_name"])["type"]
school_types_df = school_types.to_frame("School Type")
school_types_df

In [ ]:
#Total students
TStudents = school_data_complete.groupby('school_name').size()
TStudents_df = TStudents.to_frame("Total Students")
TStudents_df

In [ ]:
#Total school budget
per_school_budget = school_data_complete.groupby(["school_name"]).mean()["budget"]
per_school_budget_df = per_school_budget.to_frame("Total School Budget")
per_school_budget_df

In [ ]:
#Per student budget
per_school_capita = per_school_budget / TStudents
per_school_capita_df = per_school_capita.to_frame("Per Student Budget")
per_school_capita_df

In [ ]:
#Average math score
per_school_MS = school_data_complete.groupby(["school_name"]).mean()["math_score"]
per_school_MS_df = per_school_MS.to_frame("Average Math Score")
per_school_MS_df

In [ ]:
#Average reading score
per_school_RS = school_data_complete.groupby(["school_name"]).mean()["reading_score"]
per_school_RS_df = per_school_RS.to_frame("Average Reading Score")
per_school_RS_df

In [ ]:
#% passing math (the percentage of students who passed math)
per_school_MPassing = school_data_complete[(school_data_complete["math_score"] >= 70)].groupby(["school_name"])["student_name"].size()
per_school_MPPercent = per_school_MPassing / TStudents * 100
per_school_MPPercent_df = per_school_MPPercent.to_frame("% Passing Math")
per_school_MPPercent_df

In [ ]:
#% passing reading (the percentage of students who passed reading)
per_school_RPassing = school_data_complete[(school_data_complete["reading_score"] >= 70)].groupby(["school_name"])["student_name"].size()
per_school_RPPercent = per_school_RPassing / TStudents * 100
per_school_RPPercent_df = per_school_RPPercent.to_frame("% Passing Reading")
per_school_RPPercent_df

In [ ]:
#% overall passing (the percentage of students who passed math AND reading)
per_school_AllPassing = school_data_complete[(school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)].groupby(["school_name"])["student_name"].size()
per_school_AllPPercent = per_school_AllPassing / TStudents * 100
per_school_AllPPercent_df = per_school_AllPPercent.to_frame("% Overall Passing")
per_school_AllPPercent_df

In [ ]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
index = 0
df_to_merge = [
    school_types_df, TStudents_df, per_school_budget_df, 
    per_school_capita_df, per_school_MS_df, per_school_RS_df, 
    per_school_MPPercent_df, per_school_RPPercent_df, per_school_AllPPercent_df
]

per_school_summary = df_to_merge[0]
for index in range(1, len(df_to_merge)):
    per_school_summary = per_school_summary.merge(df_to_merge[index], on='school_name')

# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)
per_school_summary = per_school_summary.sort_values(by='school_name')

# Display the DataFrame
per_school_summary

## Highest-Performing Schools (by % Overall Passing)

Sort the schools by `% Overall Passing` in descending order and display the top 5 rows. 
Save the results in a DataFrame called "top_schools".

In [ ]:
top_schools = per_school_summary.sort_values(by='% Overall Passing', ascending=False)
top_schools.head()

## Lowest-Performing Schools (by % Overall Passing)

Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
Save the results in a DataFrame called "bottom_schools".

In [ ]:
bottom_schools = per_school_summary.sort_values(by='% Overall Passing', ascending=True)
bottom_schools.head()

## Math Scores by Grade

Perform the necessary calculations to create a DataFrame that lists the average math score for students of each grade level (9th, 10th, 11th, 12th) at each school.

In [ ]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_graders_scores = ninth_graders.groupby("school_name")["math_score"].mean()
tenth_graders_scores = tenth_graders.groupby("school_name")["math_score"].mean()
eleventh_graders_scores = eleventh_graders.groupby("school_name")["math_score"].mean()
twelfth_graders_scores = twelfth_graders.groupby("school_name")["math_score"].mean()

# Use the code to select only the `math_score`.
ninth_grade_math_scores = ninth_graders_scores.to_frame("math_score")
tenth_grader_math_scores = tenth_graders_scores.to_frame("math_score")
eleventh_grader_math_scores = eleventh_graders_scores.to_frame("math_score")
twelfth_grader_math_scores = twelfth_graders_scores.to_frame("math_score")

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.concat([ninth_grade_math_scores, tenth_grader_math_scores, eleventh_grader_math_scores, twelfth_grader_math_scores], axis=1)
math_scores_by_grade.columns = ['9th', '10th', '11th', '12th']

# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade

## Reading Scores by Grade

Create a DataFrame that lists the average reading score for students of each grade level (9th, 10th, 11th, 12th) at each school.

In [ ]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_graders_scores = ninth_graders.groupby("school_name")["reading_score"].mean()
tenth_graders_scores = tenth_graders.groupby("school_name")["reading_score"].mean()
eleventh_graders_scores = eleventh_graders.groupby("school_name")["reading_score"].mean()
twelfth_graders_scores = twelfth_graders.groupby("school_name")["reading_score"].mean()

# Use the code to select only the `reading_score`.
ninth_grade_reading_scores = ninth_graders_scores.to_frame("reading_score")
tenth_grader_reading_scores = tenth_graders_scores.to_frame("reading_score")
eleventh_grader_reading_scores = eleventh_graders_scores.to_frame("reading_score")
twelfth_grader_reading_scores = twelfth_graders_scores.to_frame("reading_score")

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = pd.concat([ninth_grade_reading_scores, tenth_grader_reading_scores, eleventh_grader_reading_scores, twelfth_grader_reading_scores], axis=1)
reading_scores_by_grade.columns = ['9th', '10th', '11th', '12th']

# Minor data wrangling
reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

## Scores by School Spending

Create a table that breaks down school performance based on average spending ranges (per student).

Use the code provided below to create four bins with reasonable cutoff values to group school spending.

spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]
Use `pd.cut` to categorize spending based on the bins.

Use the following code to then calculate mean scores per spending range.

spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]
Use the scores above to create a DataFrame called `spending_summary`.

Include the following metrics in the table:

Average math score

Average reading score

% passing math (the percentage of students who passed math)

% passing reading (the percentage of students who passed reading)

% overall passing (the percentage of students who passed math AND reading)

In [ ]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["< $585","$585-630", "$630−645","$645-680"]

In [ ]:
# Create a copy of the school summary since it has the "Per Student Budget" 
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

school_spending_df = per_school_summary.copy()
school_spending_df['Per Student Budget'] = school_spending_df['Per Student Budget'].str.replace('$', '')
school_spending_df['Per Student Budget'] = school_spending_df['Per Student Budget'].astype(float)

In [ ]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df['Per Student Budget'], bins=spending_bins, labels=labels)
school_spending_df["Per Student Budget"] = school_spending_df["Per Student Budget"].map("${:,.2f}".format)
school_spending_df

In [ ]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]

In [ ]:
# Assemble into DataFrame
spending_summary = pd.concat([spending_math_scores, spending_reading_scores, spending_passing_math, spending_passing_reading, overall_passing_spending], axis=1)

# Display results
spending_summary

## Scores by School Size

Use the following code to bin the `per_school_summary`.

size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).

In [ ]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [ ]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = 

In [ ]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = per_school_summary.groupby(["School Size"]).mean()["Average Reading Score"]
size_passing_math = per_school_summary.groupby(["School Size"]).mean()["% Passing Math"]
size_passing_reading = per_school_summary.groupby(["School Size"]).mean()["% Passing Reading"]
size_overall_passing = per_school_summary.groupby(["School Size"]).mean()["% Overall Passing"]

In [ ]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`


# Display results
size_summary

## Scores by School Type

Use the `per_school_summary` DataFrame from the previous step to create a new DataFrame called `type_summary`.

This new DataFrame should show school performance based on the "School Type".